In [1]:
import sys
sys.path.append("..")

In [2]:
from resources.utils import get_puzzle_input

### Part 1

Only two recipes are on the board: the first recipe got a score of 3, the second, 7. Each of the two Elves has a current recipe: the first Elf starts with the first recipe, and the second Elf starts with the second recipe.

To create new recipes, the two Elves combine their current recipes. This creates new recipes from the digits of the sum of the current recipes' scores. With the current recipes' scores of 3 and 7, their sum is 10, and so two new recipes would be created: the first with score 1 and the second with score 0. If the current recipes' scores were 2 and 3, the sum, 5, would only create one recipe (with a score of 5) with its single digit.

The new recipes are added to the end of the scoreboard in the order they are created. So, after the first round, the scoreboard is 3, 7, 1, 0.

After all new recipes are added to the scoreboard, each Elf picks a new current recipe. To do this, the Elf steps forward through the scoreboard a number of recipes equal to 1 plus the score of their current recipe. So, after the first round, the first Elf moves forward 1 + 3 = 4 times, while the second Elf moves forward 1 + 7 = 8 times. If they run out of recipes, they loop back around to the beginning. After the first round, both Elves happen to loop around until they land on the same recipe that they had in the beginning; in general, they will move to different recipes.

Drawing the first Elf as parentheses and the second Elf as square brackets, they continue this process:

```
(3)[7]
(3)[7] 1  0 
 3  7  1 [0](1) 0 
 3  7  1  0 [1] 0 (1)
(3) 7  1  0  1  0 [1] 2 
 3  7  1  0 (1) 0  1  2 [4]
 3  7  1 [0] 1  0 (1) 2  4  5 
 3  7  1  0 [1] 0  1  2 (4) 5  1 
 3 (7) 1  0  1  0 [1] 2  4  5  1  5 
 3  7  1  0  1  0  1  2 [4](5) 1  5  8 
 3 (7) 1  0  1  0  1  2  4  5  1  5  8 [9]
 3  7  1  0  1  0  1 [2] 4 (5) 1  5  8  9  1  6 
 3  7  1  0  1  0  1  2  4  5 [1] 5  8  9  1 (6) 7 
 3  7  1  0 (1) 0  1  2  4  5  1  5 [8] 9  1  6  7  7 
 3  7 [1] 0  1  0 (1) 2  4  5  1  5  8  9  1  6  7  7  9 
 3  7  1  0 [1] 0  1  2 (4) 5  1  5  8  9  1  6  7  7  9  2 
 ```
 
The Elves think their skill will improve after making a few recipes (your puzzle input). However, that could take ages; you can speed this up considerably by identifying the scores of the ten recipes after that. For example:

If the Elves think their skill will improve after making 9 recipes, the scores of the ten recipes after the first nine on the scoreboard would be 5158916779 (highlighted in the last line of the diagram).
After 5 recipes, the scores of the next ten would be 0124515891.
After 18 recipes, the scores of the next ten would be 9251071085.
After 2018 recipes, the scores of the next ten would be 5941429882.
What are the scores of the ten recipes immediately after the number of recipes in your puzzle input?

In [11]:
def new_recipes(recipe_1, recipe_2):
    mixed = recipe_1 + recipe_2
    if mixed >= 10:
        return (mixed // 10, mixed % 10)
    else:
        return (mixed % 10, )

In [12]:
assert new_recipes(3, 7) == (1, 0)
assert new_recipes(2, 3) == (5, )

In [42]:
def score(num_turns):
    scoreboard = [3, 7]
    elf_1_pos = 0
    elf_2_pos = 1
    
    while len(scoreboard) < num_turns + 10:
        elf_1_score = scoreboard[elf_1_pos]
        elf_2_score = scoreboard[elf_2_pos]
        elf_1_pos = (elf_1_pos + 1 + elf_1_score) % len(scoreboard)
        elf_2_pos = (elf_2_pos + 1 + elf_2_score) % len(scoreboard)
        scoreboard.extend(new_recipes(scoreboard[elf_1_pos], scoreboard[elf_2_pos]))
    
    return ''.join(str(s) for s in scoreboard[num_turns:num_turns + 10])        

In [43]:
# After 5 recipes, the scores of the next ten would be 0124515891.
# After 18 recipes, the scores of the next ten would be 9251071085.
# After 2018 recipes, the scores of the next ten would be 5941429882.
assert score(9) == '5158916779'
assert score(5) == '0124515891'
assert score(18) == '9251071085'
assert score(2018) == '5941429882'


In [44]:
score(846021)

'5482326119'

### Part 2

As it turns out, you got the Elves' plan backwards. They actually want to know how many recipes appear on the scoreboard to the left of the first recipes whose scores are the digits from your puzzle input.

```
51589 first appears after 9 recipes.
01245 first appears after 5 recipes.
92510 first appears after 18 recipes.
59414 first appears after 2018 recipes.
```

How many recipes appear on the scoreboard to the left of the score sequence in your puzzle input?

In [86]:
def recipes_to_left(score_to_match): 
    score_to_match_tuple = tuple(int(char) for char in score_to_match)
    scoreboard = [3, 7]
    elf_1_pos = 0
    elf_2_pos = 1
    
    def found():
        offset = len(score_to_match_tuple)
        score_board_len = len(scoreboard)
        if score_board_len < offset:
            return 0

        # Small optimisation of only checking the whole sequence if the first two
        # numbers match. If the numbers are evenly distributed we'll only do the
        # full check 1 in 100 times
        to_the_left = score_board_len - offset
        if (
            scoreboard[-1] == score_to_match_tuple[-1] and
            scoreboard[-2] == score_to_match_tuple[-2] and
            tuple(scoreboard[-offset:]) == score_to_match_tuple
        ):
            return to_the_left

        if (
            scoreboard[-2] == score_to_match_tuple[-1] and
            scoreboard[-3] == score_to_match_tuple[-2] and
            tuple(scoreboard[-(offset + 1):-1]) == score_to_match_tuple
        ):
            return to_the_left - 1
                 
        return 0
        
    
    while not found():
        if len(scoreboard) % 1000000 == 0:
            print('Number of recipes: ', len(scoreboard))
        elf_1_score = scoreboard[elf_1_pos]
        elf_2_score = scoreboard[elf_2_pos]
        elf_1_pos = (elf_1_pos + 1 + elf_1_score) % len(scoreboard)
        elf_2_pos = (elf_2_pos + 1 + elf_2_score) % len(scoreboard)
        scoreboard.extend(new_recipes(scoreboard[elf_1_pos], scoreboard[elf_2_pos]))
        #print(scoreboard)
    
    return found()

In [87]:
# As it turns out, you got the Elves' plan backwards. They actually want to know how many recipes appear on the scoreboard to the left of the first recipes whose scores are the digits from your puzzle input.

# 51589 first appears after 9 recipes.
# 01245 first appears after 5 recipes.
# 92510 first appears after 18 recipes.
# 59414 first appears after 2018 recipes.

assert score('51589') == 9
assert score('01245') == 5
assert score('92510') == 18
assert score('59414') == 2018

In [88]:
%%time

print('Recipes to left: ', score('846021'))

Number of recipes:  4000000
Number of recipes:  5000000
Number of recipes:  6000000
Number of recipes:  7000000
Number of recipes:  8000000
Number of recipes:  9000000
Number of recipes:  10000000
Number of recipes:  11000000
Number of recipes:  12000000
Number of recipes:  14000000
Number of recipes:  15000000
Number of recipes:  16000000
Number of recipes:  17000000
Number of recipes:  19000000
Number of recipes:  20000000
Recipes to left:  20368140
CPU times: user 21.8 s, sys: 152 ms, total: 21.9 s
Wall time: 22 s
